# Python 4. Ders
- Web Scraping
    - Webbrowser
    - Requests
    - BeautifulSoup
---

---
## Webbrowser

Verilen URL'yi varsayılan tarayıcıda açmamıza yarar.


In [ ]:
import webbrowser

In [ ]:
webbrowser.open('https://www.google.com/')

In [ ]:
sorgu = input('Lütfen aramak istediğiniz adresi giriniz: ')
url = 'https://www.google.com/maps/search/' + sorgu
webbrowser.open(url)

In [ ]:
sorgu = input('Lütfen aramak istediğiniz adresi giriniz: ')

url = 'https://www.google.com/maps/?q=' + sorgu
webbrowser.open(url)

---
## Requests

Sunucuya HTTP istekleri göndermemizi sağlayan bir modüldür.

In [ ]:
import requests

In [ ]:
# requests.get(url) ile bir sunucuya HTTP isteği gönderebiliriz
resp = requests.get('https://tr.wikipedia.org/')
resp.status_code

# ya da

resp.ok

In [ ]:
# Eğer bir hata vermesini istiyorsak .raise_for_status() metodunu kullanırız
resp.raise_for_status()

In [ ]:
print(resp.url)

In [ ]:
print(resp.text)

In [ ]:
print(resp.content)

\>>> **Unicode hakkında daha detaylı bilgi için: https://bit.ly/unipain** <<<

In [ ]:
print(resp.headers)

In [ ]:
print(resp.headers['content-type'])

In [ ]:
sorgu = input('Lütfen aramak istediğiniz adresi giriniz: ')
payload = {'q':sorgu}
resp = requests.get('https://www.google.com/maps/', payload)
resp.ok

In [ ]:
webbrowser.open(resp.url)

In [ ]:
# İnternetten resim indirme uygulaması
resp = requests.get('https://upload.wikimedia.org/wikipedia/commons/thumb/1/1c/Galata_Tower_after_the_2020_restoration.jpg/800px-Galata_Tower_after_the_2020_restoration.jpg')
resp.ok

In [ ]:
resp.headers['content-type']

In [ ]:
with open('galata_kulesi.jpeg', 'wb') as image:
    image.write(resp.content)

In [ ]:
# Belleği aşırı yüklememek için parça parça indirebiliriz
with open('galata_kulesi.jpeg', 'wb') as image:
    for chunk in resp.iter_content(1024):
        image.write(chunk)

Bazı yaygın durum kodları:
- `200`: OK (İstek başarılı bir şekilde karşılandı) 
- `404`: Not Found  (Aranan dosya bulunamadı)
- `400`: Bad Request (Kötü istek, genelde sözdizimi hatasından kaynaklanır)
- `503`: Service Unavailable (Sunucunun bir sebepten isteği yerine getiremediğini belirtir.)

Daha fazlası için: https://developer.mozilla.org/en-US/docs/Web/HTTP/Status

---
## BeautifulSoup

Sunucudan aldığımız HTML'i kolayca ayrıştırmamıza yarayan bir kütüphanedir.

[BeautifulSoup dokümentasyonu](https://www.crummy.com/software/BeautifulSoup/bs4/doc/)

In [ ]:
import bs4
from urllib.parse import urljoin # İki url'i birleştirmek için kullanıyoruz.

In [ ]:
url = 'https://tr.wikipedia.org/'
resp = requests.get(url)
resp.ok

In [ ]:
soup = bs4.BeautifulSoup(resp.text)

In [ ]:
anchor_tags = soup.find_all('a')
print(anchor_tags)

In [ ]:
anchor_tags = soup.find_all('a', href=True)

In [ ]:
# Bütün anchor etiketlerini bul ve barındırdığı linkleri, içindeki text ile beraber yazdır.
for a in anchor_tags:
    if a['href'].startswith('/'):
        # Eğer href'ler / ile başlıyorsa başına https://tr.wikipedia.org/ adresini koyarak gösteriyoruz.
        print(a.contents[0], ':', urljoin(url, a.get('href')))
    else:
        print(a.contents[0], ':', a.get('href'))

In [ ]:
# Amazon fiyat kontrol etme
import re


def getAmazonPrice(amazon_url):
    'Request a response, then check for errors'
    res = requests.get(amazon_url)
    res.raise_for_status()

    soup = bs4.BeautifulSoup(res.text, features='html.parser')
    elems = soup.select('#price_inside_buybox')

    return elems[0].text.strip()


url_regex = re.compile(r'(.*)\??')
url = url_regex.search(input('Input the url: ')).group(1)
print('the url is ', url)
print('The price is', getAmazonPrice(url))